In [1]:
import sys
import cv2
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
#https://github.com/Yunyung/Bayesian-Matting-implementation

In [2]:
file_name = 'horse'
img = cv2.imread('./dog.bmp', cv2.IMREAD_COLOR)
img = img/255
trimap = cv2.imread('./dog_trim.bmp', cv2.IMREAD_GRAYSCALE)

fg_mask = trimap == 255
bg_mask = trimap == 0
unknown_mask = True ^ np.logical_or(fg_mask, bg_mask)

In [3]:
alpha = np.zeros(trimap.shape)
alpha[fg_mask] = 1
alpha[unknown_mask] = np.nan 
fg = img*np.repeat(fg_mask[:, :, np.newaxis], 3, axis=2) 
bg = img*np.repeat(bg_mask[:, :, np.newaxis], 3, axis=2)

In [4]:
unknown = np.argwhere(np.isnan(alpha))

In [5]:
PASS = 0
total = len(unknown)

In [6]:
def dilationUnknownFgBgNeighbor(unknown_mask, kernal_size, fg_mask, bg_mask):

    kernel = np.ones((kernal_size,kernal_size),np.uint8)
    dilation_alpha = cv2.dilate(unknown_mask, kernel, iterations = 1)
    
    dila_fg_mask = np.logical_and(fg_mask, dilation_alpha)
    dila_bg_mask = np.logical_and(bg_mask, dilation_alpha)
    
    return dila_fg_mask, dila_bg_mask

In [7]:
while len(unknown) > 0:
        y,x = unknown[0]
        #print("Progress:{}, y = {}, x = {}, img = {}".format(PASS + 1, y, x, img[y][x]))
        
        #########################################
        # TODO: prepare data points and apply GMM


        # Dilation parameters used for calling cv2.dilation()
        initKernal_size = 80
        iterIncreKernal = 80
        # Perpare data points -> the surrounding pixels of the unknown part 
        # loop and find until the fg and bg points are enough to apply GMM
        kernal_size = initKernal_size
        while True: 
            temp_alpha = np.zeros(trimap.shape)
            temp_alpha[y, x] = 1 # assgin only one unknown points to mask for dilation
            dila_fg_mask, dila_bg_mask = dilationUnknownFgBgNeighbor(temp_alpha, kernal_size, fg_mask, bg_mask) # get fg mask, bg mask after dilation process 
            
            if (np.count_nonzero(dila_fg_mask) >= 30 and np.count_nonzero(dila_bg_mask) >= 30):
                print("kernal_size={}\n".format(kernal_size))
                break
            kernal_size += iterIncreKernal # if not enough, increase the filter size for dilation process
        dila_fg = img * np.repeat(dila_fg_mask[:, :, np.newaxis], 3, axis=2)
        dila_bg = img * np.repeat(dila_bg_mask[:, :, np.newaxis], 3, axis=2)
        dila_fg = dila_fg.reshape(dila_fg.shape[0]*dila_fg.shape[1], 3)
        dila_bg = dila_bg.reshape(dila_bg.shape[0]*dila_bg.shape[1], 3)
        fg_reduced = dila_fg[~np.all(dila_fg == 0, axis=1)] # exclude the datapoints they aren't the surrounding pixels of the unknown part 
        bg_reduced = dila_bg[~np.all(dila_bg == 0, axis=1)] 
            

        # Apply GMM on fg and bg points
        num_fg_components = 3
        num_bg_components = 3
        gmm_max_iter = 100
        gmm_n_init = 5
        fg_gmm = GaussianMixture(n_components=num_fg_components, max_iter=gmm_max_iter, n_init=gmm_n_init).fit(fg_reduced)
        bg_gmm = GaussianMixture(n_components=num_bg_components, max_iter=gmm_max_iter, n_init=gmm_n_init).fit(bg_reduced)
        
        # extract the inforamtion of trained GMM to apply optimization
        inv_cov_fgmm = np.linalg.inv(fg_gmm.covariances_)
        inv_cov_bgmm = np.linalg.inv(bg_gmm.covariances_)
        mean_fgmm = fg_gmm.means_
        mean_bgmm = bg_gmm.means_

        #########################################
    
        ##########################################################################
        # TODO: try to set different initial of alpha and optimize F, B, and alpha
        
        C = img[y][x]
        sigmaC = 0.01
        initAlpha = 0.1
        iterNum = 30
        invSigmaSqr = 1 / (sigmaC**2)
        I = np.eye(3)
        maxlike_pair = np.zeros(2)
        max_F = np.zeros(3)
        max_B = np.zeros(3)
        max_a = np.zeros(0)
        maxlike = -np.inf
        minlike = 1e-6

        # Optimization F, G, Alpha to each pair of fg and bg, then get the maximum likelihood
        for i in range(num_fg_components):
            mu_Fi = mean_fgmm[i]
            invSigma_Fi = inv_cov_fgmm[i]
            for j in range(num_bg_components):
                mu_Bj = mean_bgmm[j]
                invSigma_Bj = inv_cov_bgmm[j]

                F = []
                B = []
                a = initAlpha
                lastlike = -np.inf # record the last likelihood value for early stop
                for it in range(iterNum):
                    # Update F and B, fix a,  
                    A11 = invSigma_Fi + I * (a**2) * invSigmaSqr
                    A12 = I * a * (1 - a) * invSigmaSqr
                    A22 = invSigma_Bj + I * ((1 - a) ** 2) *  invSigmaSqr
                    A = np.vstack( (np.hstack((A11, A12)), np.hstack((A12, A22))) )
                    b1 = invSigma_Fi @ mu_Fi + C * a * invSigmaSqr
                    b2 = invSigma_Bj @ mu_Bj + C * (1 - a) * invSigmaSqr
                    b = np.concatenate((b1, b2))
                    X = np.linalg.solve(A, b)
                    
                    F = np.maximum(0, np.minimum(1, X[0:3]))
                    B = np.maximum(0, np.minimum(1, X[3:6]))
                    
                    # Update alpha, fix F, B
                    a = np.maximum(0, np.minimum(1, np.dot((C - B), (F - B)) / (np.linalg.norm(F - B)**2)))                
                    
                    # calculate likelihood
                    L_C = -((np.linalg.norm(C - a * F - (1 - a) * B)**2) * invSigmaSqr)
                    L_F = (- ((F - mu_Fi) @ invSigma_Fi @ (F - mu_Fi)) / 2)
                    L_B = (- ((B - mu_Bj) @ invSigma_Bj @ (B - mu_Bj)) / 2)
                    like = L_C + L_F + L_B
                    
                    if maxlike < like:
                        maxlike = like
                        #maxlike_pair = [i, j] # record i, j index
                        max_F = F
                        max_B = B
                        max_a = a
                        
                    # early stop, if have been converge 
                    if abs(like - lastlike) <= minlike:
                        break
                        
                    lastlike = like
        
        alpha[y][x] = max_a
        fg[y][x] = max_F
        bg[y][x] = max_B
        
        
                
        ##########################################################################
        
        unknown = np.delete(unknown, 0, 0)      
        PASS += 1
        sys.stdout.write("\rprogress:\t{}/{}\n".format(PASS,total))
        sys.stdout.flush()

kernal_size=80

progress:	1/29275
kernal_size=80

progress:	2/29275
kernal_size=80

progress:	3/29275
kernal_size=80

progress:	4/29275
kernal_size=80

progress:	5/29275
kernal_size=80

progress:	6/29275
kernal_size=80

progress:	7/29275
kernal_size=80

progress:	8/29275
kernal_size=80

progress:	9/29275
kernal_size=80

progress:	10/29275
kernal_size=80

progress:	11/29275
kernal_size=80

progress:	12/29275
kernal_size=80

progress:	13/29275
kernal_size=80

progress:	14/29275
kernal_size=80

progress:	15/29275
kernal_size=80

progress:	16/29275
kernal_size=80

progress:	17/29275
kernal_size=80

progress:	18/29275
kernal_size=80

progress:	19/29275
kernal_size=80

progress:	20/29275
kernal_size=80

progress:	21/29275
kernal_size=80

progress:	22/29275
kernal_size=80

progress:	23/29275
kernal_size=80

progress:	24/29275
kernal_size=80

progress:	25/29275
kernal_size=80

progress:	26/29275
kernal_size=80

progress:	27/29275
kernal_size=80

progress:	28/29275
kernal_size=80

progress:	29/

kernal_size=80

progress:	232/29275
kernal_size=80

progress:	233/29275
kernal_size=80

progress:	234/29275
kernal_size=80

progress:	235/29275
kernal_size=80

progress:	236/29275
kernal_size=80

progress:	237/29275
kernal_size=80

progress:	238/29275
kernal_size=80

progress:	239/29275
kernal_size=80

progress:	240/29275
kernal_size=80

progress:	241/29275
kernal_size=80

progress:	242/29275
kernal_size=80

progress:	243/29275
kernal_size=80

progress:	244/29275
kernal_size=80

progress:	245/29275
kernal_size=80

progress:	246/29275
kernal_size=80

progress:	247/29275
kernal_size=80

progress:	248/29275
kernal_size=80

progress:	249/29275
kernal_size=80

progress:	250/29275
kernal_size=80

progress:	251/29275
kernal_size=80

progress:	252/29275
kernal_size=80

progress:	253/29275
kernal_size=80

progress:	254/29275
kernal_size=80

progress:	255/29275
kernal_size=80

progress:	256/29275
kernal_size=80

progress:	257/29275
kernal_size=80

progress:	258/29275
kernal_size=80

progress:	25

kernal_size=80

progress:	460/29275
kernal_size=80

progress:	461/29275
kernal_size=80

progress:	462/29275
kernal_size=80

progress:	463/29275
kernal_size=80

progress:	464/29275
kernal_size=80

progress:	465/29275
kernal_size=80

progress:	466/29275
kernal_size=80

progress:	467/29275
kernal_size=80

progress:	468/29275
kernal_size=80

progress:	469/29275
kernal_size=80

progress:	470/29275
kernal_size=80

progress:	471/29275
kernal_size=80

progress:	472/29275
kernal_size=80

progress:	473/29275
kernal_size=80

progress:	474/29275
kernal_size=80

progress:	475/29275
kernal_size=80

progress:	476/29275
kernal_size=80

progress:	477/29275
kernal_size=80

progress:	478/29275
kernal_size=80

progress:	479/29275
kernal_size=80

progress:	480/29275
kernal_size=80

progress:	481/29275
kernal_size=80

progress:	482/29275
kernal_size=80

progress:	483/29275
kernal_size=80

progress:	484/29275
kernal_size=80

progress:	485/29275
kernal_size=80

progress:	486/29275
kernal_size=80

progress:	48

kernal_size=80

progress:	688/29275
kernal_size=80

progress:	689/29275
kernal_size=80

progress:	690/29275
kernal_size=80

progress:	691/29275
kernal_size=80

progress:	692/29275
kernal_size=80

progress:	693/29275
kernal_size=80

progress:	694/29275
kernal_size=80

progress:	695/29275
kernal_size=80

progress:	696/29275
kernal_size=80

progress:	697/29275
kernal_size=80

progress:	698/29275
kernal_size=80

progress:	699/29275
kernal_size=80

progress:	700/29275
kernal_size=80

progress:	701/29275
kernal_size=80

progress:	702/29275
kernal_size=80

progress:	703/29275
kernal_size=80

progress:	704/29275
kernal_size=80

progress:	705/29275
kernal_size=80

progress:	706/29275
kernal_size=80

progress:	707/29275
kernal_size=80

progress:	708/29275
kernal_size=80

progress:	709/29275
kernal_size=80

progress:	710/29275
kernal_size=80

progress:	711/29275
kernal_size=80

progress:	712/29275
kernal_size=80

progress:	713/29275
kernal_size=80

progress:	714/29275
kernal_size=80

progress:	71

kernal_size=80

progress:	916/29275
kernal_size=80

progress:	917/29275
kernal_size=80

progress:	918/29275
kernal_size=80

progress:	919/29275
kernal_size=80

progress:	920/29275
kernal_size=80

progress:	921/29275
kernal_size=80

progress:	922/29275
kernal_size=80

progress:	923/29275
kernal_size=80

progress:	924/29275
kernal_size=80

progress:	925/29275
kernal_size=80

progress:	926/29275
kernal_size=80

progress:	927/29275
kernal_size=80

progress:	928/29275
kernal_size=80

progress:	929/29275
kernal_size=80

progress:	930/29275
kernal_size=80

progress:	931/29275
kernal_size=80

progress:	932/29275
kernal_size=80

progress:	933/29275
kernal_size=80

progress:	934/29275
kernal_size=80

progress:	935/29275
kernal_size=80

progress:	936/29275
kernal_size=80

progress:	937/29275
kernal_size=80

progress:	938/29275
kernal_size=80

progress:	939/29275
kernal_size=80

progress:	940/29275
kernal_size=80

progress:	941/29275
kernal_size=80

progress:	942/29275
kernal_size=80

progress:	94

...

kernal_size=80

progress:	28360/29275
kernal_size=80

progress:	28361/29275
kernal_size=80

progress:	28362/29275
kernal_size=80

progress:	28363/29275
kernal_size=80

progress:	28364/29275
kernal_size=80

progress:	28365/29275
kernal_size=80

progress:	28366/29275
kernal_size=80

progress:	28367/29275
kernal_size=80

progress:	28368/29275
kernal_size=80

progress:	28369/29275
kernal_size=80

progress:	28370/29275
kernal_size=80

progress:	28371/29275
kernal_size=80

progress:	28372/29275
kernal_size=80

progress:	28373/29275
kernal_size=80

progress:	28374/29275
kernal_size=80

progress:	28375/29275
kernal_size=80

progress:	28376/29275
kernal_size=80

progress:	28377/29275
kernal_size=80

progress:	28378/29275
kernal_size=80

progress:	28379/29275
kernal_size=80

progress:	28380/29275
kernal_size=80

progress:	28381/29275
kernal_size=80

progress:	28382/29275
kernal_size=80

progress:	28383/29275
kernal_size=80

progress:	28384/29275
kernal_size=80

progress:	28385/29275
kernal_size=

kernal_size=80

progress:	28576/29275
kernal_size=80

progress:	28577/29275
kernal_size=80

progress:	28578/29275
kernal_size=80

progress:	28579/29275
kernal_size=80

progress:	28580/29275
kernal_size=80

progress:	28581/29275
kernal_size=80

progress:	28582/29275
kernal_size=80

progress:	28583/29275
kernal_size=80

progress:	28584/29275
kernal_size=80

progress:	28585/29275
kernal_size=80

progress:	28586/29275
kernal_size=80

progress:	28587/29275
kernal_size=80

progress:	28588/29275
kernal_size=80

progress:	28589/29275
kernal_size=80

progress:	28590/29275
kernal_size=80

progress:	28591/29275
kernal_size=80

progress:	28592/29275
kernal_size=80

progress:	28593/29275
kernal_size=80

progress:	28594/29275
kernal_size=80

progress:	28595/29275
kernal_size=80

progress:	28596/29275
kernal_size=80

progress:	28597/29275
kernal_size=80

progress:	28598/29275
kernal_size=80

progress:	28599/29275
kernal_size=80

progress:	28600/29275
kernal_size=80

progress:	28601/29275
kernal_size=

kernal_size=80

progress:	28792/29275
kernal_size=80

progress:	28793/29275
kernal_size=80

progress:	28794/29275
kernal_size=80

progress:	28795/29275
kernal_size=80

progress:	28796/29275
kernal_size=80

progress:	28797/29275
kernal_size=80

progress:	28798/29275
kernal_size=80

progress:	28799/29275
kernal_size=80

progress:	28800/29275
kernal_size=80

progress:	28801/29275
kernal_size=80

progress:	28802/29275
kernal_size=80

progress:	28803/29275
kernal_size=80

progress:	28804/29275
kernal_size=80

progress:	28805/29275
kernal_size=80

progress:	28806/29275
kernal_size=80

progress:	28807/29275
kernal_size=80

progress:	28808/29275
kernal_size=80

progress:	28809/29275
kernal_size=80

progress:	28810/29275
kernal_size=80

progress:	28811/29275
kernal_size=80

progress:	28812/29275
kernal_size=80

progress:	28813/29275
kernal_size=80

progress:	28814/29275
kernal_size=80

progress:	28815/29275
kernal_size=80

progress:	28816/29275
kernal_size=80

progress:	28817/29275
kernal_size=

kernal_size=80

progress:	29008/29275
kernal_size=80

progress:	29009/29275
kernal_size=80

progress:	29010/29275
kernal_size=80

progress:	29011/29275
kernal_size=80

progress:	29012/29275
kernal_size=80

progress:	29013/29275
kernal_size=80

progress:	29014/29275
kernal_size=80

progress:	29015/29275
kernal_size=80

progress:	29016/29275
kernal_size=80

progress:	29017/29275
kernal_size=80

progress:	29018/29275
kernal_size=80

progress:	29019/29275
kernal_size=80

progress:	29020/29275
kernal_size=80

progress:	29021/29275
kernal_size=80

progress:	29022/29275
kernal_size=80

progress:	29023/29275
kernal_size=80

progress:	29024/29275
kernal_size=80

progress:	29025/29275
kernal_size=80

progress:	29026/29275
kernal_size=80

progress:	29027/29275
kernal_size=80

progress:	29028/29275
kernal_size=80

progress:	29029/29275
kernal_size=80

progress:	29030/29275
kernal_size=80

progress:	29031/29275
kernal_size=80

progress:	29032/29275
kernal_size=80

progress:	29033/29275
kernal_size=

kernal_size=80

progress:	29224/29275
kernal_size=80

progress:	29225/29275
kernal_size=80

progress:	29226/29275
kernal_size=80

progress:	29227/29275
kernal_size=80

progress:	29228/29275
kernal_size=80

progress:	29229/29275
kernal_size=80

progress:	29230/29275
kernal_size=80

progress:	29231/29275
kernal_size=80

progress:	29232/29275
kernal_size=80

progress:	29233/29275
kernal_size=80

progress:	29234/29275
kernal_size=80

progress:	29235/29275
kernal_size=80

progress:	29236/29275
kernal_size=80

progress:	29237/29275
kernal_size=80

progress:	29238/29275
kernal_size=80

progress:	29239/29275
kernal_size=80

progress:	29240/29275
kernal_size=80

progress:	29241/29275
kernal_size=80

progress:	29242/29275
kernal_size=80

progress:	29243/29275
kernal_size=80

progress:	29244/29275
kernal_size=80

progress:	29245/29275
kernal_size=80

progress:	29246/29275
kernal_size=80

progress:	29247/29275
kernal_size=80

progress:	29248/29275
kernal_size=80

progress:	29249/29275
kernal_size=

In [8]:
grayscale_imgName = 'output_dog.png'
cv2.imwrite(grayscale_imgName, alpha * 255)

True